## Importamos las librerias

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re

## Importamos los json

In [48]:
df_games = pd.read_json(r'Datasets\output_steam_games.json' ,lines=True)

In [3]:
data_list = []        # Lista para almacenar los diccionarios JSON de cada línea

file_path = r"Datasets\australian_user_reviews.json"

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_user_reviews = pd.DataFrame(data_list)

In [4]:
data_list = []   # Lista para almacenar los diccionarios JSON de cada línea

file_path = r"Datasets\australian_users_items.json"

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_user_items = pd.DataFrame(data_list)

#revisar que hace está funcion xd

## Visualizamos los Datasets

In [12]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


In [17]:
df_user_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [18]:
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Vemos que los df tienen muchos datos anidados y filas completamente vacias, vamos a trabajar sobre ello

## Desanidamos los datasets df_user_reviews y df_user_items

In [37]:
df_user_reviews_desc = df_user_reviews.explode(['reviews']) #descomponemos la columna reviews
df_reviews_series = df_user_reviews_desc['reviews'].apply(pd.Series) #creamos una serie a partir de ella
df_user_reviews = pd.concat([df_user_reviews_desc, df_reviews_series], axis=1) #creamos el dataframe definitivo 
                                                                            #y despues borramos la columna que quedó de más
df_user_reviews.drop('reviews', axis=1, inplace = True)     #al estar agregada al dataframe,ya la podemos borrar

KeyError: 'reviews'

In [9]:
df_user_items_desc = df_user_items.explode(["items"])
df_user_items_desc2 = pd.json_normalize(df_user_items_desc['items']).set_index(df_user_items_desc['items'].index)
df_user_items= pd.concat([df_user_items_desc2, df_user_items_desc], axis=1)
df_user_items.drop('items', axis=1, inplace = True)

## Borramos todas las filas que esten totalmente vacias

In [10]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 120445
la cantidad de filas de items son 5170015
la cantidad de filas de reviews son 59333


In [49]:
df_games = df_games.dropna(how ="all").reset_index(drop=True)
df_user_reviews = df_user_reviews.dropna(how ="all").reset_index(drop=True)
df_user_items = df_user_items.dropna(how ="all").reset_index(drop=True)
df_user_reviews.drop(0, axis=1, inplace = True)       #borramos la columna vacia

# ETL para df_games

Vemos los items con el mismo id en games

In [50]:
duplicated_rows_games = df_games[df_games.duplicated(subset = "id", keep=False)]
duplicated_rows_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


La primera fila hay que borrarla, ya que no contiene nada, las de bethesda son iguales, por lo que cabría eliminarlas.

In [51]:
batman_mask = df_games["title"] == "Batman: Arkham City - Game of the Year Edition"
batman = df_games[batman_mask]
batman

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Son el mismo juego, por lo que cabe eliminar la columna 

In [52]:
df_games = df_games.drop([30961,74,13894])

En df games temos tags y genres que tienen los mismos atributos (son categorias bastante parecidas). Por lo cual, para no desaprovechar, ya que nos vamos a decantar por una para la api y para el modelo machine learning, podemos aprovechar las instancias en las que genres este vacio y tags no y viceversa, y convinarlos en una columna.
Primero comprobaremos en que casos se da

In [53]:

filas_vacias_genres = df_games[df_games['genres'].isna()] 
filas_vacias_genres


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
11,None,None,Icarus Six Sixty Six,None,http://store.steampowered.com/app/724910/Icaru...,None,[Casual],http://steamcommunity.com/app/724910/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",Free,0.0,724910.0,None
19,None,None,After Life VR,None,http://store.steampowered.com/app/772590/After...,None,"[Early Access, Indie, VR]",http://steamcommunity.com/app/772590/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",4.99,1.0,772590.0,None
20,None,None,Kitty Hawk,None,http://store.steampowered.com/app/640250/Kitty...,None,"[Early Access, Action, Adventure, Indie, Casual]",http://steamcommunity.com/app/640250/reviews/?...,"[Single-player, Steam Leaderboards, HTC Vive, ...",2.99,1.0,640250.0,None
22,None,None,Mortars VR,None,http://store.steampowered.com/app/711440/Morta...,None,"[Early Access, Strategy, Action, Indie, Casual...",http://steamcommunity.com/app/711440/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.99,1.0,711440.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32118,None,None,Robotpencil Presents: Exercise: Brushwork,Robotpencil Presents: Exercise: Brushwork,http://store.steampowered.com/app/775640/Robot...,2018-01-03,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/775640/reviews/?...,None,3.99,0.0,775640.0,None
32119,None,None,Robotpencil Presents: Creative Composition,Robotpencil Presents: Creative Composition,http://store.steampowered.com/app/777930/Robot...,2018-01-03,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777930/reviews/?...,None,3.99,0.0,777930.0,None
32120,None,None,The Gamble House,The Gamble House,http://store.steampowered.com/app/775370/The_G...,2016-11-19,[Movie],http://steamcommunity.com/app/775370/reviews/?...,[Captions available],4.99,0.0,775370.0,None
32121,None,None,Kalen Chock Presents: 2017 Free Tutorial,Kalen Chock Presents: 2017 Free Tutorial,http://store.steampowered.com/app/777950/Kalen...,2018-01-03,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777950/reviews/?...,None,Free,0.0,777950.0,None


In [54]:
filas_vacias_tags = df_games[df_games['tags'].isna()] 
filas_vacias_tags

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
279,Her Interactive,[Adventure],Nancy Drew®: Ransom of the Seven Ships Demo,Nancy Drew®: Ransom of the Seven Ships Demo,http://store.steampowered.com/app/31990/Nancy_...,2009-07-21,None,http://steamcommunity.com/app/31990/reviews/?b...,"[Single-player, Game demo]",None,0.0,31990.0,Her Interactive
298,"SQUARE ENIX, Eidos Interactive","[Action, Adventure]",Mini Ninjas Demo,Mini Ninjas Demo,http://store.steampowered.com/app/35050/Mini_N...,2009-08-19,None,http://steamcommunity.com/app/35050/reviews/?b...,"[Single-player, Game demo]",None,0.0,35050.0,IO Interactive
309,Freeze Tag Inc.,[Casual],Freeze Tag Fun Pack #2,Freeze Tag Fun Pack #2,http://store.steampowered.com/app/39392/Freeze...,2009-09-23,None,http://steamcommunity.com/app/39392/reviews/?b...,[Single-player],19.99,0.0,39392.0,"Freeze Tag Inc.,iQ212"
310,Freeze Tag Inc.,[Casual],Freeze Tag Fun Pack #1,Freeze Tag Fun Pack #1,http://store.steampowered.com/app/39391/Freeze...,1999-11-30,None,http://steamcommunity.com/app/39391/reviews/?b...,[Single-player],19.99,0.0,39391.0,"Freeze Tag Inc.,Joju Games,Linksolutions Ltd.,..."
358,Paradox Interactive,None,Europa Universalis III: Heir to the Throne,Europa Universalis III: Heir to the Throne,http://store.steampowered.com/app/25806/Europa...,2009-12-15,None,http://steamcommunity.com/app/25806/reviews/?b...,"[Single-player, Multi-player, Downloadable Con...",9.99,0.0,25806.0,Paradox Interactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31625,PlayFirst,[Casual],Zenerchi®,Zenerchi®,http://store.steampowered.com/app/37290/Zenerchi/,2009-08-12,None,http://steamcommunity.com/app/37290/reviews/?b...,[Single-player],9.99,0.0,37290.0,Red Rocket Games
31627,PlayFirst,[Simulation],Wedding Dash® 2: Rings Around the World,Wedding Dash® 2: Rings Around the World,http://store.steampowered.com/app/37280/Weddin...,2009-08-12,None,http://steamcommunity.com/app/37280/reviews/?b...,[Single-player],9.99,0.0,37280.0,Sarbakan
31639,Sandlot Games,None,Super Granny Collection,Super Granny Collection,http://store.steampowered.com/app/36270/Super_...,2009-07-17,None,http://steamcommunity.com/app/36270/reviews/?b...,[Single-player],14.99,0.0,36270.0,Sandlot Games
31652,Strategy First,[Indie],Bad Rats: the Rats' Revenge Demo,Bad Rats: the Rats' Revenge Demo,http://store.steampowered.com/app/34910/Bad_Ra...,2009-07-20,None,http://steamcommunity.com/app/34910/reviews/?b...,"[Single-player, Game demo]",None,0.0,34910.0,Invent4 Entertainment


Ahora haremos una nueva columna con los valores de genres, rellando los faltantes con tags y viceversa 

In [55]:

df_games['genre'] = df_games['genres'].combine_first(df_games['tags'])

df_games = df_games.drop(['tags', 'genres'], axis=1)
df_games


,publisher,app_name,title,url,release_date,reviews_url,specs,price,early_access,id,developer,genre
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]"
1,"Making Fun, Inc.",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]"
2,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]"
3,彼岸领域,弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,"[Action, Adventure, Casual]"
4,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,"[Action, Indie, Casual, Sports]"
...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS""","[Casual, Indie, Simulation, Strategy]"
32131,Sacada,LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,"[Casual, Indie, Strategy]"
32132,Laush Studio,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,"[Indie, Racing, Simulation]"
32133,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns","[Casual, Indie]"


Desanidamos las fila genre, borramos las columnas url, reviews_url y specs, ya que no las vamos a utilizar, de df_games 

In [56]:
df_games = df_games.explode(["genre"])
df_games = df_games.drop(['specs', 'url', 'reviews_url'], axis=1)

In [59]:
df_games

,publisher,app_name,title,release_date,price,early_access,id,developer,genre
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Action
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Casual
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Indie
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Simulation
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Strategy
...,...,...,...,...,...,...,...,...,...
32134,None,Maze Run VR,None,None,4.99,1.0,681550.0,None,Adventure
32134,None,Maze Run VR,None,None,4.99,1.0,681550.0,None,Indie
32134,None,Maze Run VR,None,None,4.99,1.0,681550.0,None,Action
32134,None,Maze Run VR,None,None,4.99,1.0,681550.0,None,Simulation


## Borramos los duplicados

In [33]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 2088545
la cantidad de filas de items son 5170015
la cantidad de filas de reviews son 59333


In [34]:
df_games = df_games.drop_duplicates()
df_user_reviews = df_user_reviews.drop_duplicates()
df_user_items = df_user_items.drop_duplicates()

In [35]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 2088545
la cantidad de filas de items son 5110819
la cantidad de filas de reviews son 58459
